Create surrogate neural population responses during natural behavior trials using reduced populations with parameters chosen based on theoretical considerations.

In [1]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import signal, stats
from sklearn import linear_model
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import get_seg
from disp import set_font_size
from my_stats import nanpearsonr

from record import smlt_ppln_ma

cc = np.concatenate

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS

Load behavioral data from Coen et al 2014

In [2]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
df_behav

,ID,FRAME,T,Q,S,P,F,FFV,FFA,FLS,FRS,FMFV,FMLS
0,0,0,0.000000,1,0,0,0,0.231,0.0000,0.159,0.077,-0.058526,0.274257
1,0,1,0.033300,1,0,0,0,0.314,0.0385,0.025,0.044,0.100623,0.298490
2,0,2,0.066600,1,0,0,0,0.308,0.0025,0.029,0.011,0.147629,0.271865
3,0,3,0.099900,1,0,0,0,0.319,0.0020,0.016,0.016,0.091174,0.306112
4,0,4,0.133200,1,0,0,0,0.312,-0.0315,0.030,0.002,0.065871,0.306439
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4978565,275,27100,902.430902,1,0,0,0,0.217,0.0280,0.029,0.058,-0.171418,0.136184
4978566,275,27101,902.464202,1,0,0,0,0.210,-0.1045,0.138,0.322,0.032522,0.249171
4978567,275,27102,902.497502,1,0,0,0,0.008,-0.0910,0.034,0.260,0.033202,0.010844
4978568,275,27103,902.530803,1,0,0,0,0.028,0.0695,0.010,0.090,0.012059,0.027177


In [3]:
# split big df into dfs for individual trials
n_tr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in range(n_tr)]

tr_lens = np.array([len(df_tr) for df_tr in dfs_tr])
tr_lens_cum = cc([[0], np.cumsum(tr_lens)])

n_t_total = np.sum(tr_lens)
assert n_t_total == tr_lens_cum[-1]
df_behav = None

# Baseline MA model params

In [4]:
# neural response fits from Baker data
df_ma_param = pd.read_csv('data/simple/neur/baker_ma_fit_param.csv')
df_ma_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,30.336758,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,21.821295,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,15.058055,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,11.820576,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,55.496220,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,1440.994269,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,27.643238,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,6.155551,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,90.705238,2.390705,0.604586


# Pulse-only selectivity, fixed $\tau_r = 120 s$, range of $\tau_a$ from 0.3 s to 1 s.

In [6]:
SAVE_DIR = 'data/simple/mlv/ma_thry/ma_fast_adapt_slow_decay_pulse_only'
SAVE_PFX = 'mlv_ma_fast_adapt_slow_decay_pulse_only'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

N = 8

df_data = {
    'EXPT_ID': np.arange(N),
    'TAU_R': 120,
    'TAU_A': np.linspace(0.3, 1, N),
    'X_S': 0,
    'X_P': 1,
}

df_ma_thry = pd.DataFrame(columns=['EXPT_ID', 'TAU_R', 'TAU_A', 'X_S', 'X_P'], data=df_data)
df_ma_thry

In [7]:
expt_ids = df_ma_thry['EXPT_ID']

tau_rs = np.array(df_ma_thry['TAU_R']).astype(float)
tau_as = np.array(df_ma_thry['TAU_A']).astype(float)
x_ss = np.array(df_ma_thry['X_S']).astype(float)
x_ps = np.array(df_ma_thry['X_P']).astype(float)

In [8]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate population response
    rs = smlt_ppln_ma(i_s, i_p, tau_rs, tau_as, x_ss, x_ps, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

....................................................................................................................................................................................................................................................................................

# Pure selectivity (pulse and sine), fixed $\tau_r = 120 s$, range of $\tau_a$ from 0.3 s to 1 s.

In [7]:
SAVE_DIR = 'data/simple/mlv/ma_thry/ma_fast_adapt_slow_decay'
SAVE_PFX = 'mlv_ma_fast_adapt_slow_decay'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
N = 16

df_data = {
    'EXPT_ID': np.arange(N),
    'TAU_R': 120,
    'TAU_A': cc([np.linspace(0.3, 1, N//2), np.linspace(0.3, 1, N//2)]),
    'X_S': cc([np.repeat(0, N//2), np.repeat(1, N//2)]),
    'X_P': cc([np.repeat(1, N//2), np.repeat(0, N//2)]),
}

df_ma_thry = pd.DataFrame(columns=['EXPT_ID', 'TAU_R', 'TAU_A', 'X_S', 'X_P'], data=df_data)
df_ma_thry

,EXPT_ID,TAU_R,TAU_A,X_S,X_P
0,0,120,0.3,0,1
1,1,120,0.4,0,1
2,2,120,0.5,0,1
3,3,120,0.6,0,1
4,4,120,0.7,0,1
5,5,120,0.8,0,1
6,6,120,0.9,0,1
7,7,120,1.0,0,1
8,8,120,0.3,1,0
9,9,120,0.4,1,0


In [8]:
expt_ids = df_ma_thry['EXPT_ID']

tau_rs = np.array(df_ma_thry['TAU_R']).astype(float)
tau_as = np.array(df_ma_thry['TAU_A']).astype(float)
x_ss = np.array(df_ma_thry['X_S']).astype(float)
x_ps = np.array(df_ma_thry['X_P']).astype(float)

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate population response
    rs = smlt_ppln_ma(i_s, i_p, tau_rs, tau_as, x_ss, x_ps, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

.......................................................

# Pure selectivity (pulse and sine), fixed $\tau_r = 120 s$, range of $\tau_a$ from 0.3 s to 1 s.

In [11]:
SAVE_DIR = 'data/simple/mlv/ma_thry/ma_dense_fast_adapt_slow_decay'
SAVE_PFX = 'mlv_ma_dense_fast_adapt_slow_decay'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
N = 48

df_data = {
    'EXPT_ID': np.arange(N),
    'TAU_R': 120,
    'TAU_A': cc([np.linspace(0.3, 1, N//2), np.linspace(0.3, 1, N//2)]),
    'X_S': cc([np.repeat(0, N//2), np.repeat(1, N//2)]),
    'X_P': cc([np.repeat(1, N//2), np.repeat(0, N//2)]),
}

df_ma_thry = pd.DataFrame(columns=['EXPT_ID', 'TAU_R', 'TAU_A', 'X_S', 'X_P'], data=df_data)
df_ma_thry

,EXPT_ID,TAU_R,TAU_A,X_S,X_P
0,0,120,0.300000,0,1
1,1,120,0.330435,0,1
2,2,120,0.360870,0,1
3,3,120,0.391304,0,1
4,4,120,0.421739,0,1
5,5,120,0.452174,0,1
6,6,120,0.482609,0,1
7,7,120,0.513043,0,1
8,8,120,0.543478,0,1
9,9,120,0.573913,0,1


In [12]:
expt_ids = df_ma_thry['EXPT_ID']

tau_rs = np.array(df_ma_thry['TAU_R']).astype(float)
tau_as = np.array(df_ma_thry['TAU_A']).astype(float)
x_ss = np.array(df_ma_thry['X_S']).astype(float)
x_ps = np.array(df_ma_thry['X_P']).astype(float)

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate population response
    rs = smlt_ppln_ma(i_s, i_p, tau_rs, tau_as, x_ss, x_ps, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

.......................................................

# Pure selectivity (pulse and sine), fixed $\tau_r = 120 s$, range of $\tau_a = $ 0.3 or 1s.

In [17]:
SAVE_DIR = 'data/simple/mlv/ma_thry/ma_fast_adapt_0.3_1_slow_decay'
SAVE_PFX = 'mlv_ma_fast_adapt_0.3_1_slow_decay'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
N = 4

df_data = {
    'EXPT_ID': np.arange(N),
    'TAU_R': 120,
    'TAU_A': cc([np.linspace(0.3, 1, N//2), np.linspace(0.3, 1, N//2)]),
    'X_S': cc([np.repeat(0, N//2), np.repeat(1, N//2)]),
    'X_P': cc([np.repeat(1, N//2), np.repeat(0, N//2)]),
}

df_ma_thry = pd.DataFrame(columns=['EXPT_ID', 'TAU_R', 'TAU_A', 'X_S', 'X_P'], data=df_data)
df_ma_thry

,EXPT_ID,TAU_R,TAU_A,X_S,X_P
0,0,120,0.3,0,1
1,1,120,1.0,0,1
2,2,120,0.3,1,0
3,3,120,1.0,1,0


In [18]:
expt_ids = df_ma_thry['EXPT_ID']

tau_rs = np.array(df_ma_thry['TAU_R']).astype(float)
tau_as = np.array(df_ma_thry['TAU_A']).astype(float)
x_ss = np.array(df_ma_thry['X_S']).astype(float)
x_ps = np.array(df_ma_thry['X_P']).astype(float)

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate population response
    rs = smlt_ppln_ma(i_s, i_p, tau_rs, tau_as, x_ss, x_ps, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

.......................................................

# Variable selectivity, fixed $\tau_r = 120 s$, range of $\tau_a$ from 0.3 s to 1 s.

In [9]:
SAVE_DIR = 'data/simple/mlv/ma_thry/ma_fast_adapt_slow_decay_var_select'
SAVE_PFX = 'mlv_ma_fast_adapt_slow_decay_var_select'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
N = 32

df_data = {
    'EXPT_ID': np.arange(N),
    'TAU_R': 120,
    'TAU_A': cc([np.linspace(0.3, 1, N//4), np.linspace(0.3, 1, N//4), np.linspace(0.3, 1, N//4), np.linspace(0.3, 1, N//4)]),
    'X_S': cc([np.repeat(0, N//4), np.repeat(.25, N//4), np.repeat(.75, N//4), np.repeat(1, N//4)]),
    'X_P': cc([np.repeat(1, N//4), np.repeat(.75, N//4), np.repeat(.25, N//4), np.repeat(0, N//4)]),
}

df_ma_thry = pd.DataFrame(columns=['EXPT_ID', 'TAU_R', 'TAU_A', 'X_S', 'X_P'], data=df_data)
df_ma_thry

,EXPT_ID,TAU_R,TAU_A,X_S,X_P
0,0,120,0.3,0.00,1.00
1,1,120,0.4,0.00,1.00
2,2,120,0.5,0.00,1.00
3,3,120,0.6,0.00,1.00
4,4,120,0.7,0.00,1.00
5,5,120,0.8,0.00,1.00
6,6,120,0.9,0.00,1.00
7,7,120,1.0,0.00,1.00
8,8,120,0.3,0.25,0.75
9,9,120,0.4,0.25,0.75


In [10]:
expt_ids = df_ma_thry['EXPT_ID']

tau_rs = np.array(df_ma_thry['TAU_R']).astype(float)
tau_as = np.array(df_ma_thry['TAU_A']).astype(float)
x_ss = np.array(df_ma_thry['X_S']).astype(float)
x_ps = np.array(df_ma_thry['X_P']).astype(float)

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate population response
    rs = smlt_ppln_ma(i_s, i_p, tau_rs, tau_as, x_ss, x_ps, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

.......................................................